In [16]:
import json

import pandas as pd

In [17]:
import sys
import os

# Get the current working directory and the relative path to your module
module_path = os.path.abspath(os.path.join("..", ".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from src.api.service.repo import SearchRepo

In [18]:
es = SearchRepo()

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '9I9q3mFYSISbvBg75cK23A',
 'name': '5eeda383872f',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


In [19]:
idx="test"

In [20]:
with open("mapping/settings.json", "r") as file:
    settings = json.load(file)["settings"]

with open("mapping/mappings.json", "r") as file:
    mappings = json.load(file)["mappings"]

In [7]:
resp = es.create_index(index=idx, mappings=mappings, settings=settings,)

Index 'test' created successfully.


# 1. Test Analyzers

In [8]:
df = pd.read_json("./data/processed_vehicle_machine_data.json",  orient='records')
doc = df.iloc[0]
doc


description    Experience the thrill of driving with this 200...
facets         {'type': 'Vehicle', 'category': 'Coupe', 'bran...
Name: 0, dtype: object

In [9]:
vehicle_text = ['Experience the thrill of driving with this 2008 Lotus Exige S, a budget-friendly sports coupe that combines lightweight design with exhilarating performance. With its striking red exterior and agile handling, this vehicle promises an unforgettable ride. Perfect for enthusiasts seeking a unique driving experience without breaking the bank.',
 'Vehicle Coupe Lotus Exige S 2008 I4 1.8L Manual RWD Gasoline Red Standard Standard Leather seats Navigation Bluetooth ABS Airbags Stability control']

In [21]:
text = ['experience', 'experiences', 'experiment', 'experiments']

In [23]:
resp = es.es.indices.analyze(
    index=idx,
    analyzer="custom_standard",
    text=text,
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'experience'},
  {'token': 'experience'},
  {'token': 'experiment'},
  {'token': 'experiment'}]}

In [24]:
resp = es.es.indices.analyze(
    index=idx,
    analyzer="english",
    text=text,
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'experi'},
  {'token': 'experi'},
  {'token': 'experi'},
  {'token': 'experi'}]}

In [25]:
resp = es.es.indices.analyze(
    index=idx,
    analyzer="standard",
    text=text,
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'experience'},
  {'token': 'experiences'},
  {'token': 'experiment'},
  {'token': 'experiments'}]}

In [32]:
text = ["mousepad", "mouse pad"]

In [33]:
resp = es.es.indices.analyze(
    index=idx,
    analyzer="shingles",
    text=text,
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'mousepad'},
  {'token': 'mouse'},
  {'token': 'mousepad'},
  {'token': 'pad'}]}

In [34]:
resp = es.es.indices.analyze(
    text=vehicle_text[0],
    index=idx,
    analyzer="shingles",
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'experience'},
  {'token': 'experiencethe'},
  {'token': 'the'},
  {'token': 'thethrill'},
  {'token': 'thrill'},
  {'token': 'thrillof'},
  {'token': 'of'},
  {'token': 'ofdriving'},
  {'token': 'driving'},
  {'token': 'drivingwith'},
  {'token': 'with'},
  {'token': 'withthis'},
  {'token': 'this'},
  {'token': 'this2008'},
  {'token': '2008'},
  {'token': '2008lotus'},
  {'token': 'lotus'},
  {'token': 'lotusexige'},
  {'token': 'exige'},
  {'token': 'exiges'},
  {'token': 's'},
  {'token': 'sa'},
  {'token': 'a'},
  {'token': 'abudget'},
  {'token': 'budget'},
  {'token': 'budgetfriendly'},
  {'token': 'friendly'},
  {'token': 'friendlysports'},
  {'token': 'sports'},
  {'token': 'sportscoupe'},
  {'token': 'coupe'},
  {'token': 'coupethat'},
  {'token': 'that'},
  {'token': 'thatcombines'},
  {'token': 'combines'},
  {'token': 'combineslightweight'},
  {'token': 'lightweight'},
  {'token': 'lightweightdesign'},
  {'token': 'design'},
  {'token': 'designwith'}

In [13]:
text = "mouse pad"

resp = es.es.indices.analyze(
    index=idx,
    analyzer="shingles",
    text=text,
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'mouse'}, {'token': 'mousepad'}, {'token': 'pad'}]}

In [15]:
resp = es.es.indices.analyze(
    index=idx, tokenizer="num_w_unit_tokens", text="1250 cm", filter_path="**.token"
)
resp.body

{'tokens': [{'token': '1250 cm'}]}

In [118]:
analyzer = "num_attributes"

resp = es.es.indices.analyze(index=_index, analyzer=analyzer, text=text[0])
resp.body

{'tokens': [{'token': '2008lotus',
   'start_offset': 43,
   'end_offset': 53,
   'type': 'word',
   'position': 0},
  {'token': '.a',
   'start_offset': 61,
   'end_offset': 64,
   'type': 'word',
   'position': 1},
  {'token': '.with',
   'start_offset': 156,
   'end_offset': 162,
   'type': 'word',
   'position': 2},
  {'token': '.this',
   'start_offset': 207,
   'end_offset': 213,
   'type': 'word',
   'position': 3},
  {'token': '.perfect',
   'start_offset': 252,
   'end_offset': 261,
   'type': 'word',
   'position': 4}]}

In [69]:
analyzer = ["standard", "english", 'minimal', "custom_standard", "shingles"]

output = {}
for a in analyzer:
    resp = es.es.indices.analyze(index=_index, analyzer=a, text=text)
    output[a]=resp.body

In [70]:
output

{'standard': {'tokens': [{'token': 'experience',
    'start_offset': 0,
    'end_offset': 10,
    'type': '<ALPHANUM>',
    'position': 0},
   {'token': 'the',
    'start_offset': 11,
    'end_offset': 14,
    'type': '<ALPHANUM>',
    'position': 1},
   {'token': 'thrill',
    'start_offset': 15,
    'end_offset': 21,
    'type': '<ALPHANUM>',
    'position': 2},
   {'token': 'of',
    'start_offset': 22,
    'end_offset': 24,
    'type': '<ALPHANUM>',
    'position': 3},
   {'token': 'driving',
    'start_offset': 25,
    'end_offset': 32,
    'type': '<ALPHANUM>',
    'position': 4},
   {'token': 'with',
    'start_offset': 33,
    'end_offset': 37,
    'type': '<ALPHANUM>',
    'position': 5},
   {'token': 'this',
    'start_offset': 38,
    'end_offset': 42,
    'type': '<ALPHANUM>',
    'position': 6},
   {'token': '2008',
    'start_offset': 43,
    'end_offset': 47,
    'type': '<NUM>',
    'position': 7},
   {'token': 'lotus',
    'start_offset': 48,
    'end_offset': 53,
    

In [57]:
analyzer = "english"

resp = es.es.indices.analyze(analyzer=analyzer, text=text)
resp.body

{'tokens': [{'token': 'experi',
   'start_offset': 0,
   'end_offset': 10,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'thrill',
   'start_offset': 15,
   'end_offset': 21,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'drive',
   'start_offset': 25,
   'end_offset': 32,
   'type': '<ALPHANUM>',
   'position': 4},
  {'token': '2008',
   'start_offset': 43,
   'end_offset': 47,
   'type': '<NUM>',
   'position': 7},
  {'token': 'lotu',
   'start_offset': 48,
   'end_offset': 53,
   'type': '<ALPHANUM>',
   'position': 8},
  {'token': 'exig',
   'start_offset': 54,
   'end_offset': 59,
   'type': '<ALPHANUM>',
   'position': 9},
  {'token': 's',
   'start_offset': 60,
   'end_offset': 61,
   'type': '<ALPHANUM>',
   'position': 10},
  {'token': 'budget',
   'start_offset': 65,
   'end_offset': 71,
   'type': '<ALPHANUM>',
   'position': 12},
  {'token': 'friendli',
   'start_offset': 72,
   'end_offset': 80,
   'type': '<ALPHANUM>',
   'position': 13},
  {'token': '

In [58]:
analyzer = "standard"

resp = es.es.indices.analyze(analyzer=analyzer, text=text)
resp.body

{'tokens': [{'token': 'experience',
   'start_offset': 0,
   'end_offset': 10,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'the',
   'start_offset': 11,
   'end_offset': 14,
   'type': '<ALPHANUM>',
   'position': 1},
  {'token': 'thrill',
   'start_offset': 15,
   'end_offset': 21,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'of',
   'start_offset': 22,
   'end_offset': 24,
   'type': '<ALPHANUM>',
   'position': 3},
  {'token': 'driving',
   'start_offset': 25,
   'end_offset': 32,
   'type': '<ALPHANUM>',
   'position': 4},
  {'token': 'with',
   'start_offset': 33,
   'end_offset': 37,
   'type': '<ALPHANUM>',
   'position': 5},
  {'token': 'this',
   'start_offset': 38,
   'end_offset': 42,
   'type': '<ALPHANUM>',
   'position': 6},
  {'token': '2008',
   'start_offset': 43,
   'end_offset': 47,
   'type': '<NUM>',
   'position': 7},
  {'token': 'lotus',
   'start_offset': 48,
   'end_offset': 53,
   'type': '<ALPHANUM>',
   'position': 8},
  {'token': 'ex

In [59]:
analyzer = "english"

resp = es.es.indices.analyze(analyzer=analyzer, text="mousepad")
resp.body

{'tokens': [{'token': 'mousepad',
   'start_offset': 0,
   'end_offset': 8,
   'type': '<ALPHANUM>',
   'position': 0}]}

Indexing

Index 'vehicle' created successfully.
5000 documents inserted into 'vehicle'
Index with 5000 documents created in 20657665 milliseconds.


In [38]:
resp.body

{'errors': False,
 'took': 20324030,
 'ingest_took': 108,
 'items': [{'index': {'_index': 'vehicle',
    '_id': 'GqNHdpMBKlUM0qVBICG-',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 0,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'vehicle',
    '_id': 'G6NHdpMBKlUM0qVBICG-',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'vehicle',
    '_id': 'HKNHdpMBKlUM0qVBICG-',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 2,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'vehicle',
    '_id': 'HaNHdpMBKlUM0qVBICG-',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 3,
    '_primary_term': 1,
    'status': 

In [63]:
resp = es.search(index="vehicle", size=1)

Search results from 'vehicle' retrieved successfully in 3 ms.


In [64]:
resp.body

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}